In [1]:
import pyspark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1646978441780_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
%%configure -f
{"executorMemory": "4G"}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1646978441780_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1646978441780_0002,pyspark,idle,Link,Link,None,✔


In [13]:
review = spark.read.parquet("s3://ash-proj/Parquet/yelp_academic_dataset_review/*.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
review.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|y7Ssng5Bnl75QiZUE...|   0|2016-03-06 08:49:08|    0|lyjloY7uVRsaOZObr...|  4.0|Ever wanted too j...|     0|Vu1P84ifxzFeQ9qRF...|
|4fzjAjzLCnjWfQP1s...|   0|2013-12-04 22:37:18|    0|GfPzojSq1cin_fOdf...|  5.0|Dry-aged bone-in ...|     0|J1n84Z7tRI8Y7nhFU...|
|KZEYzpFSC25cuuclg...|   0|2011-03-08 22:00:44|    0|lFphcMI6ZN2uFA8wg...|  5.0|I lived in Boston...|     2|uWHsX45RpNabc9X4x...|
|pwejc1NUK6cQz7aK_...|   1|2007-11-24 22:38:04|    0|oQDDv49nI55lHzsI3...|  3.0|Some good things ...|     0|iDA5qOj534am_AacK...|
|sRNcBkZrs2bOrijfE...|   0|2013-11-08 17:40:00|    0|78ZDs404elxc0f1i-...|  4.0|Great plac

In [5]:
review.withColumn("Agg",col("")* -1)
    .show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

StructType(List(StructField(business_id,StringType,true),StructField(cool,LongType,true),StructField(date,StringType,true),StructField(funny,LongType,true),StructField(review_id,StringType,true),StructField(stars,DoubleType,true),StructField(text,StringType,true),StructField(useful,LongType,true),StructField(user_id,StringType,true)))

In [15]:
review = review.drop("business_id","date","review_id","user_id")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
review.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['cool', 'funny', 'stars', 'text', 'useful']

In [19]:
review.select(['cool', 'funny', 'stars', 'useful']).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8635403

In [20]:
df = review.drop("text")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
import pyspark.sql.functions as F
df_zero = df.select([F.count(F.when(df[c] != 0, c)).alias(c) for c in df.columns])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
df_zero.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------+-------+-------+
|   cool|  funny|  stars| useful|
+-------+-------+-------+-------+
|1948423|1596435|8635403|3877235|
+-------+-------+-------+-------+

In [36]:
df_cool = df_zero.collect()[0][0]
df_cool

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1948423

In [37]:
df_funny = df_zero.collect()[0][1]
df_funny

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1596435

In [39]:
df_useful = df_zero.collect()[0][3]
df_useful

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3877235

In [40]:
df_stars = df_zero.collect()[0][2]
df_stars

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8635403

In [44]:
wg_cool = df_cool/df_stars
wg_cool = 1-wg_cool
wg_cool

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.7743680289153847

In [45]:
wg_funny = df_funny/df_stars
wg_funny = 1-wg_funny
wg_funny

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8151290680932899

In [46]:
wg_useful = df_useful/df_stars
wg_useful = 1-wg_useful
wg_useful

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.5510070578061035

In [ ]:
# binarize rating
def convert_rating(rating):
    rating = int(rating)
    if rating < 2: return 1
    else: return 0
    

# udf
punct_remover = udf(lambda x: remove_punct(x))
rating_convert = udf(lambda x: convert_rating(x))
review_df = review.select('review_id', punct_remover('text'), rating_convert('stars'))
review_df = review_df.withColumnRenamed('<lambda>(text)', 'text')\
                     .withColumn('label', review_df["<lambda>(stars)"].cast(IntegerType()))\
                     .drop('<lambda>(stars)')\
                     .limit(5000)